In [3]:
import numpy as np
from bitcoinlib.services.services import Service
import hashlib

In [40]:
from datetime import datetime

In [5]:
import hashlib

In [6]:
srv = Service()
block = srv.getrawblock(511114)

In [25]:
bhash = block[0:160]
s =bytes.fromhex(bhash)
s

b'\x00\x00\x00 \xdf\x03\x96\xcd\xa6\x9e\xd8\x8c\x9c\x01\xe3G{\x07\xd5\x83\x95\x8cJ\x01\xb2#H\x00\x00\x00\x00\x00\x00\x00\x00\x00\xbf\xc2\xd35\x18\t\x04:\xa7 \x80\x07rB\x9a8\xa1\xf2\x94\xceK\x08\xee\x85=+\xb0\x88F,\x82\xa7\xcc\x06\x95Z\xdc\x97]\x17\xf2E\xdc\x10'

In [28]:
s.find(b'\x06')

69

In [11]:
proc = hashlib.sha256(hashlib.sha256(binascii.unhexlify(bhash)).digest()).digest()[::-1]
proc

b'\x00\x00\x00\x00\x00\x00\x00\x00\x00ZC\xa7\t\xfa\xed\x11\xe13\x1a\x18\xb6\xaa\x1d\xba\xe7\x0f@IF\xdf\xee\xef'

In [1]:
def reverseHex(value):
    bArray = bytes.fromhex(value)[::-1].hex()
    return bArray

In [11]:
def littleEndian(value):
    bArray = bytes.fromhex(value)[::-1].hex()
    return bArray

In [29]:
def getVariableInt(value):
    if(int(value[0:2], 16) < int("fd", 16)):
        return value[0:2]
    elif(int(value[0:2], 16) == int("fd", 16)):
        return value[2:6]
    elif(int(value[0:2], 16) == int("fe", 16)):
        return value[2:10]
    elif(int(value[0:2], 16) == int("ff", 16)):
        return value[2:18]
   

In [30]:
def isNotFlag(value):
    return bool(int(value, 16))

In [14]:
offset = 0
blockJSON = {}
blockJSON["blockheader"]={}
transactions = []
blockJSON["blockheader"]["version"] = littleEndian(block[offset:offset+8]); offset+=8
blockJSON["blockheader"]["prev_block"] = littleEndian(block[offset:offset+64]); offset+=64
blockJSON["blockheader"]["merkle_root"] = littleEndian(block[offset:offset+64]); offset+=64
blockJSON["blockheader"]["timestamp"] = littleEndian(block[offset:offset+8]); offset+=8
blockJSON["blockheader"]["bits"] = littleEndian(block[offset:offset+8]); offset+=8
blockJSON["blockheader"]["nonce"] = littleEndian(block[offset:offset+8]); offset+=8
blockJSON["tx_cnt"] = littleEndian(getVariableInt(block[offset:offset+18]))
offset = offset+len(blockJSON["tx_cnt"]) if len(blockJSON["tx_cnt"]) <= 2 else offset+len(blockJSON["tx_cnt"])+2

In [31]:
tx_cnt = blockJSON["tx_cnt"]
tx_cnt

'0716'

In [20]:
block[176:200]

'010100000000000000000000'

In [13]:
for m in range(int(blockJSON["tx_cnt"], 16)):
    tx={}
    tx["inputs"] =[]
    tx["outputs"] =[]
    tx["version"] = littleEndian(block[offset:offset+8]); offset+=8
    tx["flag"] = False if isNotFlag(block[offset:offset+2]) else True
    offset = offset+4 if tx["flag"] else offset
    tx["tx_in_cnt"] = littleEndian(getVariableInt(block[offset:offset+18]))
    offset = offset+len(tx["tx_in_cnt"]) if len(tx["tx_in_cnt"]) <= 2 else offset+len(tx["tx_in_cnt"])+2

    for i in range(int(tx["tx_in_cnt"], 16)):
        tx_in = {}
        outpoint={}
        outpoint["hash"] = littleEndian(block[offset:offset+64]); offset+=64
        outpoint["index"] = littleEndian(block[offset:offset+8]); offset+=8
        tx_in["prev_out"] = outpoint
        tx_in["script_len"] = littleEndian(getVariableInt(block[offset:offset+18]))
        offset = offset+len(tx_in["script_len"]) if len(tx_in["script_len"]) <= 2 else offset+len(tx_in["script_len"])+2
        tx_in["sig_script"] = block[offset: offset+(int(tx_in["script_len"], 16)*2)]; offset+= (int(tx_in["script_len"], 16)*2)
        tx_in["sequence"] = block[offset:offset+8];offset+=8
        tx_in["witness"]=""
        tx["inputs"].append(tx_in)

    tx["tx_out_cnt"] = littleEndian(getVariableInt(block[offset:offset+18]))
    offset = offset+len(tx["tx_out_cnt"]) if len(tx["tx_out_cnt"]) <= 2 else offset+len(tx["tx_out_cnt"])+2

    for i in range(int(tx["tx_out_cnt"], 16)):    
        tx_out = {}
        tx_out["value"] = block[offset:offset+16]; offset+=16
        tx_out["pk_script_len"] = littleEndian(getVariableInt(block[offset:offset+18]))
        offset = offset+len(tx_out["pk_script_len"]) if len(tx_out["pk_script_len"]) <= 2 else offset+len(tx_out["pk_script_len"])+2
        tx_out["pk_script"] = block[offset: offset+(int(tx_out["pk_script_len"], 16)*2)]; offset+= (int(tx_out["pk_script_len"], 16)*2)
        tx["outputs"].append(tx_out)

    if(tx["flag"]):
        for j in range(int(tx["tx_in_cnt"], 16)):
            witness_str = ""
            tx_witness_cnt = littleEndian(getVariableInt(block[offset:offset+18]))
            witness_str += littleEndian(tx_witness_cnt)
            offset = offset+len(tx_witness_cnt) if len(tx_witness_cnt) <= 2 else offset+len(tx_witness_cnt)+2
            for i in range(int(tx_witness_cnt, 16)):
                wit_len = littleEndian(getVariableInt(block[offset:offset+18]))
                witness_str+=littleEndian(wit_len)
                offset = offset+len(wit_len) if len(wit_len) <= 2 else offset+len(wit_len)+2
                wit_comp = block[offset: offset+(int(wit_len, 16)*2)]; offset+= (int(wit_len, 16)*2)
                witness_str+= wit_comp
            
            tx["inputs"][j]["witness"] = witness_str

    tx["lock_time"] = littleEndian(block[offset:offset+8]); offset+=8
    transactions.append(tx)



In [10]:
blockJSON

{'blockheader': {'version': '21cce000',
  'prev_block': '00000000000000000002170d4b2aab21130dded8db8ba94f56adc1ca2e7b3490',
  'merkle_root': '09e9f79632a969777bf7539958fe75deb9e1ec8def23076befe768ff87387e80',
  'timestamp': '61a5cc10',
  'bits': '170c9a13',
  'nonce': '03ec146d'},
 'tx_cnt': '0b1e'}

In [17]:
pk_script = bytes.fromhex(transactions[0]['outputs'][0]["pk_script"])


In [55]:

pk_script

NameError: name 'pk_script' is not defined

In [54]:
l = hashlib.sha256(pk_script).digest()[::-1]

NameError: name 'pk_script' is not defined

In [40]:
l

b'\xf1\xdd\xcc\x1e\xf4\xb2\xe4\x1a\xd4K\x86_xp\x05F7}\xc3\x84.\x91\xf4"\x9b\x82\xfe4\x9e<!n'

In [42]:
''.join(('%02x' % ord(a)) for a in l.iterbytes())

AttributeError: 'bytes' object has no attribute 'iterbytes'

In [43]:
dt =datetime.fromtimestamp(int(blockJSON["blockheader"]["timestamp"], 16))

In [47]:
dt.strftime("%d-%m-%Y")

'27-02-2018'

In [145]:
for i in range(int(tx["tx_in_cnt"], 16)):
    tx_in = {}
    outpoint={}
    outpoint["hash"] = littleEndian(block[offset:offset+64]); offset+=64
    outpoint["index"] = littleEndian(block[offset:offset+8]); offset+=8
    tx_in["prev_out"] = outpoint
    tx_in["script_len"] = littleEndian(getVariableInt(block[offset:offset+18]))
    offset = offset+len(tx_in["script_len"]) if len(tx_in["script_len"]) <= 2 else offset+len(tx_in["script_len"])+2
    tx_in["sig_script"] = block[offset: offset+(int(tx_in["script_len"], 16)*2)]; offset+= (int(tx_in["script_len"], 16)*2)
    tx_in["sequence"] = block[offset:offset+8];offset+=8
    tx["inputs"].append(tx_in)

In [146]:
tx["tx_out_cnt"] = littleEndian(getVariableInt(block[offset:offset+18]))
offset = offset+len(tx["tx_out_cnt"]) if len(tx["tx_out_cnt"]) <= 2 else offset+len(tx["tx_out_cnt"])+2

In [147]:
tx["tx_out_cnt"]

'02'

In [148]:
for i in range(int(tx["tx_out_cnt"], 16)):    
    tx_out = {}
    tx_out["value"] = block[offset:offset+16]; offset+=16
    tx_out["pk_script_len"] = littleEndian(getVariableInt(block[offset:offset+18]))
    offset = offset+len(tx_out["pk_script_len"]) if len(tx_out["pk_script_len"]) <= 2 else offset+len(tx_out["pk_script_len"])+2
    tx_out["pk_script"] = block[offset: offset+(int(tx_out["pk_script_len"], 16)*2)]; offset+= (int(tx_out["pk_script_len"], 16)*2)
    tx["outputs"].append(tx_out)

In [149]:
tx["inputs"][0]["sig_script"]

'038acc0741d6a541b444b37241d6a541b32893932f4254432e544f502ffabe6d6d6c7ca42c4f7df4e0f29df2abc12894fd3e86bf4318315c00a080aeed081062fd8000000000000000cb009e2991da010000000000'

In [35]:
if(tx["flag"]):
    for i in range(int(tx["tx_in_cnt"], 16)):
        witness_str = ""
        tx_witness_cnt = littleEndian(getVariableInt(block[offset:offset+18]))
        witness_str += littleEndian(tx_witness_cnt)
        offset = offset+len(tx_witness_cnt) if len(tx_witness_cnt) <= 2 else offset+len(tx_witness_cnt)+2
        for i in range(int(tx_witness_cnt, 16)):
            wit_len = littleEndian(getVariableInt(block[offset:offset+18]))
            witness_str+=littleEndian(wit_len)
            offset = offset+len(wit_len) if len(wit_len) <= 2 else offset+len(wit_len)+2
            wit_comp = block[offset: offset+(int(wit_len, 16)*2)]; offset+= (int(wit_len, 16)*2)
            witness_str+= wit_comp
            tx["inputs"][i]["witness"] = witness_str

In [36]:
tx

{'inputs': [{'prev_out': {'hash': '0000000000000000000000000000000000000000000000000000000000000000',
    'index': 'ffffffff'},
   'script_len': '55',
   'sig_script': '038acc0741d6a541b444b37241d6a541b32893932f4254432e544f502ffabe6d6d6c7ca42c4f7df4e0f29df2abc12894fd3e86bf4318315c00a080aeed081062fd8000000000000000cb009e2991da010000000000',
   'sequence': 'ffffffff',
   'witness': '01200000000000000000000000000000000000000000000000000000000000000000'}],
 'outputs': [{'value': 'd111904f00000000',
   'pk_script_len': '19',
   'pk_script': '76a914ba507bae8f1643d2556000ca26b9301b9069dc6b88ac'},
  {'value': '0000000000000000',
   'pk_script_len': '26',
   'pk_script': '6a24aa21a9edb89df5739294fd833d0f192e14b3e96ad64711dfeb9e6c8b75cf3fdb5b4d7f15'}],
 'witness': [],
 'version': '00000001',
 'flag': True,
 'tx_in_cnt': '01',
 'tx_out_cnt': '02'}

In [37]:
tx["lock_time"] = littleEndian(block[offset:offset+8]); offset+=8

In [153]:
tx["lock_time"]

'00000000'

In [140]:
block[598:700]

'200000000000000000000000000000000000000000000000000000000000000000000000000200000004ff6a380b52aa7e56b7'